In [26]:
import pandas as pd
import hopsworks
import joblib
import datetime
#import dataframe_image as df1
from datetime import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/39062
Connected. Call `.close()` to terminate connection gracefully.


In [27]:
mr = project.get_model_registry()
model = mr.get_model('bank_note', version=1)
model_dir = model.download()
model = joblib.load(model_dir + '/bank_note_model.pkl')

Connected. Call `.close()` to terminate connection gracefully.

In [28]:
feature_view = fs.get_feature_view(name='bank_notes_dataframe', version=1)

In [29]:
batch_data = feature_view.get_batch_data()
predictions = model.predict(batch_data)

2023-05-08 16:25:26,448 INFO: USE `bwhiz_featurestore`
2023-05-08 16:25:28,956 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


In [30]:
predictions

array([1, 1, 1, ..., 0, 0, 0])

In [31]:
batch_data

,variance,skewness,curtosis,entropy
0,0.224320,-0.521470,-0.403860,1.201700
1,-2.298700,-5.227000,5.630000,0.917220
2,-4.504600,-5.812600,10.886700,-0.528460
3,3.966300,10.168400,-4.113100,-4.605600
4,-2.280400,-0.306260,1.334700,1.376300
...,...,...,...,...
1356,-1.047577,-5.638628,7.738022,1.519871
1357,-1.172324,-9.811866,5.027221,-6.114997
1358,3.353563,-5.234440,7.741583,-5.210451
1359,0.597557,5.446506,2.656908,1.679064


In [32]:
# prediciton on the last bank note sent :
latest_bank_note = predictions[predictions.size - 1]

In [33]:
latest_bank_note

0

In [34]:
bank_note_fg = fs.get_feature_group(name='bank_notes_dataframe', version=1)
df = bank_note_fg.read()
df

2023-05-08 16:25:43,885 INFO: USE `bwhiz_featurestore`
2023-05-08 16:25:44,997 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`, `fg0`.`output` `output`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


,variance,skewness,curtosis,entropy,output
0,0.224320,-0.521470,-0.403860,1.201700,1
1,-2.298700,-5.227000,5.630000,0.917220,1
2,-4.504600,-5.812600,10.886700,-0.528460,1
3,3.966300,10.168400,-4.113100,-4.605600,0
4,-2.280400,-0.306260,1.334700,1.376300,1
...,...,...,...,...,...
1356,-1.047577,-5.638628,7.738022,1.519871,0
1357,-1.172324,-9.811866,5.027221,-6.114997,0
1358,3.353563,-5.234440,7.741583,-5.210451,1
1359,0.597557,5.446506,2.656908,1.679064,0


In [35]:
### getting the actual label of the last row of data:
label = df.iloc[-1]['output']
label

1.0

In [36]:
monitor_fg = fs.get_or_create_feature_group(name = 'bank_note_predictions',
                                           version = 1,
                                           primary_key = ['datetime'],
                                           description = 'bank note prediction/output Monitoring')

In [ ]:
now = datetime.now().strftime("%m/%d/%y, %H:%M:%S")

data = {
    'prediction' : [int(latest_bank_note)],
    'label' : [label],
    'datetime' :[now]
}

monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/39062/jobs/named/bank_note_predictions_1_offline_fg_backfill/executions


In [ ]:
history_df = monitor_fg.read()
history_df

In [ ]:
df_recent = history_df.tail(5)

In [ ]:
#df1.export(df_recent, 'assets/df_recent.png') #save an image of the dataframe's last five output

In [ ]:
predictions = history_df['prediction']
labels = history_df['label']

results = confusion_matrix(labels, predictions)
print(results)

In [ ]:
script_dir = os.getcwd()

In [ ]:
assets_dir = os.path.join(script_dir, '..', 'assets')

In [ ]:
assets_dir

In [ ]:
if not os.path.exists(assets_dir):
    os.makedirs(assets_dir)

In [ ]:
try:
    if results.shape == (2,2):
        cm = sns.heatmap(results, annot=True, fmt='',
            linecolor='k',linewidths=1,cmap='Blues',cbar=False)
        
        fig = cm.get_figure()
        fig.savefig(os.path.join(assets_dir, 'confusion_matrix.png'))
        results
    else:
        print('The confusion isnt a square matrix yet , run the feature pipeline with BACKFILL == False')
except:
    pass